General description: these are the primary donation data analyzed in the paper, "Examining charitable giving in real-world online donations" 
published in Nature Communications in 2019 by Sisco and Weber. Each row represents one donation. 

-------Variables:

amount_donated - the amount donated in US dollars.


campaign_ID - an encrypted identifier for the campaign_ID. You can use this to tell which donations were made to the same campaign but not to trace the raw data back to the unencrypted campaign_ID. The encryption is to protect the privacy of the campaign creators and donors.


category - the category of each campaign.


anonymous - whether or not the donation was made anonymously to the public.


gender - The gender of the current donor. "F" represents female and "M" represents male. These genders were estimated based on the public names of the donors. The algorithm used is provided in the supplementary materials for the paper.

same_last_name - "1" means that the donor and the recipient had the same last name. "0" means that they did not (from what we could infer based on publicly displayed names). 

empathy - "1" means that an expression of empathy was detected in the message left by the current donor. "0" means that no expression of empathy was detected. The algorithm used is provided in the supplementary materials for the paper.


<h3 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>Task 1. Data Overview</strong></h3>

In [15]:
# Import the libraries
import pandas as pd
import numpy as np

### Overviewing the Data
* Dimension of the dataset, i.e. the number of rows and columns
* What are the attributes?
* What is the data type of each attribute? And what is its range?

In [4]:
# read csv
df = pd.read_csv('donor_data.csv')
print(f'Dimension of the dataset: number of rows: {df.shape[0]}, number of columns: {df.shape[1]}')
print(f'Informations of the dataset: {df.info()}')

Dimension of the dataset: number of rows: 11999, number of columns: 7
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   amount_donated  11999 non-null  int64  
 1   campaign_ID     11999 non-null  object 
 2   category        11894 non-null  object 
 3   anonymous       11999 non-null  int64  
 4   gender          7133 non-null   object 
 5   same_last_name  11825 non-null  float64
 6   empathy         11999 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 656.3+ KB
Informations of the dataset: None


In [5]:
df.head(3)

,amount_donated,campaign_ID,category,anonymous,gender,same_last_name,empathy
0,50,f3F1j8SbZZZIR/7Y9r8XUS2JwH4JNnU9zxa967TSlCAz,Medical,0,F,0.0,0
1,100,f3F1j8SbZZZIR/7Y9r8XUS2JwH4JNnU9zxa967TSlCAz,Medical,0,F,0.0,1
2,10,f3F1j8SbZZZIR/7Y9r8XUS2JwH4JNnU9zxa967TSlCAz,Medical,0,M,0.0,0


In [6]:
df.tail(3)

,amount_donated,campaign_ID,category,anonymous,gender,same_last_name,empathy
11996,25,O6D6pB+cgIR2I4snh43BNi6WyGYPNHU=,Community,0,F,0.0,0
11997,50,O6D6pB+cgIR2I4snh43BNi6WyGYPNHU=,Community,1,NaN,0.0,0
11998,25,O6D6pB+cgIR2I4snh43BNi6WyGYPNHU=,Community,0,F,0.0,0


### --------> OBSERVATION

+ We only have the `amount_donated` is a continuous numerical column 

+ I can see since the `describe` function assume all columns are categorical, it only summarizes all by count, unique, top, and frequency. Let's convert columns to their appropriate data type

In [7]:
# describe 
df.amount_donated.describe()

count    11999.000000
mean        94.636553
std        204.020417
min          2.000000
25%         25.000000
50%         50.000000
75%        100.000000
max       8980.000000
Name: amount_donated, dtype: float64

In [8]:
# Number of unique values in each column
df.nunique()

amount_donated    169
campaign_ID       299
category           20
anonymous           2
gender              2
same_last_name      2
empathy             2
dtype: int64

<h3 style="color:#ffc0cb;font-size:50px;font-family:Georgia;text-align:center;"><strong>Task 2. Auditing and cleansing the loaded data</strong></h3>

Identify data problems, fix them, and record the errors found and fixings in an error list.

1. **Syntactical Anomalies**:
anomalies concerning the format and values used for representation of the entities. Common syntactic anomalies are: lexical errors, domain format errors, syntactical error and irregularities.
2. **Semantic Anomalies**:
anomalies hindering the data collection from being a comprehensive and non-redundant representation of the mini-world. These types of anomalies include: integrity constraint violations, contradictions, duplicates and invalid tuples.
3. **Coverage Anomalies**:
anomalies decreasing the amount of entities and entity properties from the mini-world that are represented in the data collection. Coverage anomalies are categorized as: missing values and missing tuples.

#### Creating an error recorder
To record an error, we need:
- `indexOfdf`: index of the record/row in the original dataset
- `Id`: the id of the job advertisement that has the data issue. If the data issue involves all job records, just put “ALL”.
- `ColumnName`: the name of the column that the data issue locates
- `Original`: the original value of the cell
- `Modified`: the modified value of the cell
- `ErrorType` : the type of errors, for example, Missing Values, Violation of Integrity Constraint, Outliers, or any other errors found
- `Fixing`: describe how this problem was fixed.

In [9]:
# create and error recorder (i.e. the error_list)
itemlist = ['indexOfdf','Id','ColumnName', 'Original', 'Modified', 'ErrorType','Fixing']
error_list = pd.DataFrame(columns=itemlist)
error_list

,indexOfdf,Id,ColumnName,Original,Modified,ErrorType,Fixing


In [10]:
# Function for updating the error records

# update error list by attributes
def update_error_list(indexOfdf, Id, ColumnName, Orignal, Modified, ErrorType, Fixing):
    errItem=[indexOfdf, Id, ColumnName, Orignal, Modified, ErrorType,Fixing]
    error_list.loc[len(error_list)]=errItem

# Obtain all indexes of the records in the dataframe:
indices = df.index
indices

RangeIndex(start=0, stop=11999, step=1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   amount_donated  11999 non-null  int64  
 1   campaign_ID     11999 non-null  object 
 2   category        11894 non-null  object 
 3   anonymous       11999 non-null  int64  
 4   gender          7133 non-null   object 
 5   same_last_name  11825 non-null  float64
 6   empathy         11999 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 656.3+ KB


In [16]:
def coerce_df_columns_to_best_dtype(df, boolean_column_list):

    # convert to boolean dtype
    df[boolean_column_list] = df[boolean_column_list].astype('bool', errors='ignore')


    # convert object columns to string datatype
    df = df.convert_dtypes()

    # select numeric columns
    df_numeric = df.select_dtypes(include=[np.number]).columns.to_list()

    # select non-numeric columns
    df_string = df.select_dtypes(include='string').columns.tolist()

    # print out the name and number of numeric column
    print("Number of numeric columns: ", len(df_numeric))
    print("List of numeric columns: ", df_numeric, "\n")

    # print out the name and number of categorical column
    print("Number of categorical columns: ", len(df_string))
    print("List of string columns: ", df_string, "\n\n")

    # return datatype for each column after coercing
    return df.info()

# apply the coerce_df_columns_to_best_dtype function to the dataframe
coerce_df_columns_to_best_dtype(df, ['anonymous', 'same_last_name','empathy'])

# # record the problems and fixings
# update_error_list('All', 'All', 'All', 'NA', 'NA', 'Parse all columns into requuired format', 'Convert to appropriate int, float, and datetime datatype')

Number of numeric columns:  4
List of numeric columns:  ['amount_donated', 'anonymous', 'same_last_name', 'empathy'] 

Number of categorical columns:  3
List of string columns:  ['campaign_ID', 'category', 'gender'] 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   amount_donated  11999 non-null  Int64  
 1   campaign_ID     11999 non-null  string 
 2   category        11894 non-null  string 
 3   anonymous       11999 non-null  boolean
 4   gender          7133 non-null   string 
 5   same_last_name  11999 non-null  boolean
 6   empathy         11999 non-null  boolean
dtypes: Int64(1), boolean(3), string(3)
memory usage: 457.1 KB


In [10]:
# NUmber of missing values in each column
df.isnull().sum()

amount_donated       0
campaign_ID          0
category           105
anonymous            0
gender            4866
same_last_name     174
empathy              0
dtype: int64

In [37]:
# check duplication in the dataset and show duplicated rows
print(f'Number of duplicated rows: {df.duplicated().sum()}')
# show duplicated rows
df[df.duplicated()]

Number of duplicated rows: 0


,amount_donated,campaign_ID,category,anonymous,gender,same_last_name,empathy


In [39]:
# drop duplicated rows
df.drop_duplicates(inplace=True)
# check the number of rows after dropping duplicated rows
print(f'Number of rows after dropping duplicated rows: {df.shape[0]}')

Number of rows after dropping duplicated rows: 5063


In [24]:
# check speeling for similarity between rows in categorical columns
print(f'Number of unique values in the category column: {df.category.nunique()}')
print(f'Frequency of each category in descending order:\n{df.category.value_counts()}')

Number of unique values in the category column: 20
Frequency of each category in descending order:
Sandy                6273
Medical              2520
Alberta Fires         906
Charity               430
Sports                341
Family                258
Business              229
Events                213
Creative              136
Education             127
Nepal                 121
Volunteer              68
Community              62
Philippine Relief      54
Newlyweds              52
US Storms              37
Travel                 27
Faith                  19
Ecuador                15
Emergencies             6
Name: category, dtype: int64


In [30]:
# the correlation between the empathy column and the other columns
print(f'Correlation between the empathy column and the other columns:\n{df.corr()["empathy"].sort_values(ascending=False)}')

Correlation between the empathy column and the other columns:
empathy           1.000000
amount_donated    0.004352
same_last_name    0.003049
anonymous        -0.099264
Name: empathy, dtype: float64


In [44]:
# correlation between the same last name and other columns
print(f'Correlation between the last name column and the other columns:\n{df.corr()["same_last_name"].sort_values(ascending=False)}')

Correlation between the last name column and the other columns:
same_last_name    1.000000
amount_donated    0.020116
empathy           0.003049
anonymous        -0.050807
Name: same_last_name, dtype: float64


In [47]:
# Lowercase all the columns
df.columns = df.columns.str.lower()
df.head(3)

,amount_donated,campaign_id,category,anonymous,gender,same_last_name,empathy
0,50,f3F1j8SbZZZIR/7Y9r8XUS2JwH4JNnU9zxa967TSlCAz,medical,0,F,0.0,0
1,100,f3F1j8SbZZZIR/7Y9r8XUS2JwH4JNnU9zxa967TSlCAz,medical,0,F,0.0,1
2,10,f3F1j8SbZZZIR/7Y9r8XUS2JwH4JNnU9zxa967TSlCAz,medical,0,M,0.0,0
